In [1]:
import os, sys, glob
import math
import numpy as np
import pandas as pd
import networkx as nx
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
import copy
import cvxopt

In [2]:
def simulateD(nrSignatures, nrComponents=1):
    D = np.zeros(shape=(nrSignatures, nrComponents))
    for j in range(nrComponents):
        d = np.random.dirichlet(alpha=[.25]*nrSignatures)
        D[:,j] = d
        
        # remove values smaller than 0.01
        for idx, val in enumerate(D[:,j]):
            if val < 0.01:
                D[idx,j] = 0
                
        # renormalize
        total = np.sum(D[:,j])
        for idx, val in enumerate(D[:,j]):
            D[idx,j] = val / total

    return D

In [3]:
def simulatec(nrNodes, min_count, max_count):
    c = np.zeros(shape=(1, nrNodes))
    for j in range(nrNodes):
        c[0,j] = np.random.randint(low=min_count, high=max_count)
    return c

In [4]:
df_sigs = pd.read_table("../../results/signatures_cosmic.tsv", sep="\t")

In [5]:
def noise(nrContexts, nrNodes, eps):
    X = np.zeros(shape=(nrContexts, nrNodes))
    for i in range(nrContexts):
        for j in range(nrNodes):
            X[i,j] = np.random.normal(0, eps)
    return X

In [58]:
def simulate(treeFilename, df_S, k, eps):
    T = nx.read_adjlist("./m11_S109.tree")
    S = df_S.values.T
    
    V = T.nodes()
    node2idx = { node : idx for idx, node in enumerate(V) }
    
    # select k edges to remove
    E_rem = copy.copy(T.edges())
    random.shuffle(E_rem)
    E_rem = E_rem[:k]
    for e in E_rem:
        T.remove_edge(*e)
        
    components = list(nx.connected_components(T))
        
    # simulate mutation counts
    c = simulatec(len(V), 5, 100)

    nrContexts   = S.shape[0]
    nrSignatures = S.shape[1]
    nrComponents = len(components)
    
    D = simulateD(nrSignatures, nrComponents)
    df_D = pd.DataFrame(D, 
                        index=map(lambda x : "Signature." + str(x), range(1,nrSignatures+1)),
                        columns=map(lambda x : ":".join(x), components))
    
    P = np.zeros(shape=(nrContexts, len(V)))
    for idx, component in enumerate(components):
        sub_c = np.zeros(shape=(1, len(component)))
        sub_d = np.reshape(D[:, idx], (nrSignatures, 1))
        for idx2, node in enumerate(component):
            org_idx2 = node2idx[node]
            sub_c[0, idx2] = c[0, org_idx2]
        
        sub_P = np.dot(S, np.dot(sub_d, sub_c))
        for idx2, node in enumerate(component):
            P[:, node2idx[node]] = sub_P[:, idx2]
            
    P_eps = P + noise(nrContexts, len(V), eps)
    P_eps = np.maximum(np.zeros(shape=P_eps.shape), P_eps)
            
    df_P = pd.DataFrame(np.round(P).astype(int), index=df_S.columns, columns=V)
    df_P_eps = pd.DataFrame(np.round(P_eps).astype(int), index=df_S.columns, columns=V)
    
    return E_rem, df_P, df_P_eps, df_D

In [50]:
E_rem, df_P, df_P_eps, df_D = simulateP("m11_S109.tree", df_sigs, 2, .1)

In [60]:
df_P.values[:,1]

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [67]:
for filename in glob.glob("*.tree"):
    base = os.path.basename(filename).rstrip(".tree")
    for k in [0,1,2,3]:
        for eps in [.1, .5]:
            E_rem, df_P, df_P_eps, df_D = simulate(filename, df_sigs, k, eps)
            filename_E = base + "_k" + str(k) + "_eps" + str(eps) + ".E.txt"
            filename_P = base + "_k" + str(k) + "_eps" + str(eps) + ".P.txt"
            filename_P_eps = base + "_k" + str(k) + "_eps" + str(eps) +".P_eps.txt"
            filename_D = base + "_k" + str(k) + "_eps" + str(eps) + ".D.txt"
            
            with open(filename_E, "w") as f:
                for e in E_rem:
                    f.write(e[0] + " " + e[1] + "\n")
            
            df_P.to_csv(filename_P)
            df_P_eps.to_csv(filename_P_eps)
            df_D.to_csv(filename_D)

In [63]:
np.maximum(np.zeros(shape=df_P_eps.values.shape), df_P_eps.values)

array([[ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  1.,  1., ...,  0.,  0.,  1.]])

In [86]:
M = np.array([[1., 2., 0.], [-8., 3., 2.], [0., 1., 1.]])
P = np.dot(M.T, M)
q = -np.dot(M.T, np.array([3., 2., 3.]))
G = np.array([[1., 2., 1.], [2., 0., 1.], [-1., 2., -1.]])
h = np.array([3., 2., -2.]).reshape((3,))

In [296]:
def cvxopt_solve_qp(P, q, G=None, h=None, A=None, b=None):
    P = .5 * (P + P.T)  # make sure P is symmetric
    args = [cvxopt.matrix(P), cvxopt.matrix(q)]
    if G is not None:
        args.extend([cvxopt.matrix(G), cvxopt.matrix(h)])
        if A is not None:
            args.extend([cvxopt.matrix(A), cvxopt.matrix(b)])
    cvxopt.solvers.options['maxiters'] = 10000
    sol = cvxopt.solvers.qp(*args)
    if 'optimal' not in sol['status']:
        return None
    return np.array(sol['x']).reshape((P.shape[1],))

In [97]:
cvxopt_solve_qp(P, q, G, h)

     pcost       dcost       gap    pres   dres
 0: -1.0062e+01 -8.2131e+00  3e+00  8e-01  6e-17
 1: -8.9877e+00 -7.1656e+00  6e-01  3e-01  2e-16
 2: -4.7428e+00 -5.6786e+00  9e-01  1e-16  1e-15
 3: -5.5832e+00 -5.5940e+00  1e-02  5e-17  4e-16
 4: -5.5921e+00 -5.5922e+00  1e-04  2e-16  3e-16
 5: -5.5922e+00 -5.5922e+00  1e-06  1e-16  3e-16
Optimal solution found.


array([ 0.12997344, -0.06498685,  1.74005307])

In [98]:
q = np.array([[1],[2],[1]])

In [100]:
e = simulateD(30)

In [103]:
q = np.dot(S, e)

In [104]:
cvxopt_solve_qp(np.dot(S.T,S), -np.dot(S.T, q))

array([ -3.22807446e-15,   4.04016765e-16,  -4.48004064e-15,
         3.17915573e-02,   1.60108919e-14,   2.05468091e-02,
        -1.44014330e-16,  -4.73662425e-16,  -2.93142183e-16,
         5.01779342e-01,   8.43054222e-17,  -1.05866206e-15,
         1.41259844e-02,   1.56524105e-02,   5.58588205e-02,
        -6.54349033e-15,   3.38456501e-02,  -1.11276025e-15,
         7.39291068e-16,  -1.44152828e-15,  -8.79776606e-17,
         3.14858944e-16,  -3.81234148e-16,   2.96257674e-02,
         1.45818755e-02,   0.00000000e+00,  -5.78694251e-18,
         1.29599353e-01,   1.52592430e-01,  -1.26122862e-15])

In [106]:
e

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.03179156],
       [ 0.        ],
       [ 0.02054681],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.50177934],
       [ 0.        ],
       [ 0.        ],
       [ 0.01412598],
       [ 0.01565241],
       [ 0.05585882],
       [ 0.        ],
       [ 0.03384565],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.02962577],
       [ 0.01458188],
       [ 0.        ],
       [ 0.        ],
       [ 0.12959935],
       [ 0.15259243],
       [ 0.        ]])

The goal is to generate a Fixed Signature Exposure instance with feature matrix $P$ and relative exposures $d$.

In [107]:
d = simulateD(30)

We start by simulating mutation counts $c$.

In [308]:
c = simulatec(3, 1e4, 5e4)

In [309]:
c

array([[ 48364.,  42305.,  24078.]])

We generate feature matrix $P$ as $S\mathbf{d}\mathbf{c}$.

In [311]:
P = np.dot(S, np.dot(d,c)) + noise(96, 3, 0.1)

To obtain the corresponding Signature Exposure instance, we compute $\mathbf{q}$ following the reduction, i.e. $q_i = \sum_{j=1}^n c_j \cdot p_{ij}$.

In [312]:
q = np.zeros(shape=(96,1))
for i in range(96):
    for j in range(3):
        c_j = c[0, j]
        q[i,0] += c_j * P[i,j]

In [313]:
np.zeros(30)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.])

In [314]:
sum(c)

array([ 48364.,  42305.,  24078.])

In [315]:
for j in range(3):
    print np.sum(P[:,j])

48364.7553463
42304.5668687
24077.0043813


In [316]:
N = 0
for j in range(3):
    N += np.sum(P[:,j])**2

In [317]:
e_star = cvxopt_solve_qp(np.dot(S.T,S), -np.dot(S.T, q), -np.identity(30), np.zeros(30))

     pcost       dcost       gap    pres   dres
 0: -7.8772e+16 -1.1999e+17  4e+16  0e+00  1e+00
 1: -1.5569e+17 -1.8444e+17  3e+16  4e-02  3e-01
 2: -1.6645e+17 -1.9803e+17  3e+16  3e-07  1e-16
 3: -1.6889e+17 -1.7574e+17  7e+15  3e-07  1e-16
 4: -1.6947e+17 -1.7033e+17  9e+14  5e-07  2e-16
 5: -1.6958e+17 -1.6971e+17  1e+14  3e-07  1e-16
 6: -1.6961e+17 -1.6963e+17  3e+13  5e-07  1e-16
 7: -1.6961e+17 -1.6962e+17  3e+12  3e-07  1e-16
 8: -1.6961e+17 -1.6962e+17  4e+11  9e-08  1e-16
 9: -1.6961e+17 -1.6961e+17  6e+10  5e-07  1e-16
10: -1.6961e+17 -1.6961e+17  9e+09  3e-07  1e-16
11: -1.6961e+17 -1.6961e+17  1e+09  5e-07  1e-16
12: -1.6961e+17 -1.6961e+17  2e+08  3e-07  1e-16
13: -1.6961e+17 -1.6961e+17  2e+07  3e-07  1e-16
14: -1.6961e+17 -1.6961e+17  2e+06  2e-07  2e-16
15: -1.6961e+17 -1.6961e+17  7e+04  3e-07  2e-16
16: -1.6961e+17 -1.6961e+17  2e+03  3e-07  1e-16
17: -1.6961e+17 -1.6961e+17  2e+01  4e-07  2e-16
18: -1.6961e+17 -1.6961e+17  2e-01  2e-07  1e-16
19: -1.6961e+17 -1.69

In [318]:
d_star = e_star / N
# for r in range(30):
#     if d_star[r] < 1e-10:
#         d_star[r] = 0

In [319]:
np.asmatrix(d_star).T

matrix([[  2.20992533e-02],
        [  8.95300190e-57],
        [  1.83126630e-56],
        [  6.48778373e-02],
        [  5.96360778e-02],
        [  5.01761181e-05],
        [  2.15554306e-06],
        [  7.66050728e-06],
        [  3.10623537e-01],
        [  1.26318788e-58],
        [  3.95212974e-06],
        [  7.75677290e-02],
        [  5.53945588e-58],
        [  4.34877226e-58],
        [  1.09411429e-02],
        [  4.71500474e-06],
        [  8.60098913e-02],
        [  1.28039625e-05],
        [  1.87151153e-02],
        [  8.79759865e-02],
        [  1.48729716e-02],
        [  1.49006407e-02],
        [  3.16420618e-06],
        [  8.12211778e-58],
        [  4.15077556e-57],
        [  1.52046791e-06],
        [  3.45824662e-06],
        [  5.91780319e-59],
        [  1.40776460e-01],
        [  9.09182184e-02]])

In [320]:
d

array([[ 0.02211104],
       [ 0.        ],
       [ 0.        ],
       [ 0.06489338],
       [ 0.05962682],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.31062873],
       [ 0.        ],
       [ 0.        ],
       [ 0.07757486],
       [ 0.        ],
       [ 0.        ],
       [ 0.01097084],
       [ 0.        ],
       [ 0.08600981],
       [ 0.        ],
       [ 0.01872787],
       [ 0.08799176],
       [ 0.01486286],
       [ 0.0149007 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.14076784],
       [ 0.09093349]])

In [321]:
PP = np.zeros(shape=(96,1))
for i in range(96):
    for j in range(3):
        PP[i,0] += P[i,j]

In [322]:
e_star_prime = cvxopt_solve_qp(np.dot(S.T,S), -np.dot(S.T, PP), -np.identity(30), np.zeros(30))

     pcost       dcost       gap    pres   dres
 0: -4.6781e+07 -7.1293e+07  2e+07  0e+00  1e+00
 1: -9.2851e+07 -1.1063e+08  2e+07  2e-11  2e-01
 2: -9.8984e+07 -1.1706e+08  2e+07  2e-11  1e-16
 3: -1.0031e+08 -1.0421e+08  4e+06  8e-12  1e-16
 4: -1.0064e+08 -1.0113e+08  5e+05  9e-12  1e-16
 5: -1.0071e+08 -1.0079e+08  8e+04  4e-12  1e-16
 6: -1.0073e+08 -1.0074e+08  2e+04  8e-12  1e-16
 7: -1.0073e+08 -1.0073e+08  2e+03  1e-11  1e-16
 8: -1.0073e+08 -1.0073e+08  3e+02  1e-11  1e-16
 9: -1.0073e+08 -1.0073e+08  4e+01  2e-11  1e-16
Optimal solution found.


In [323]:
NN = 0
for j in range(3):
    NN += np.sum(P[:,j])

In [324]:
d_star_prime = e_star_prime / NN
# for r in range(30):
#     if d_star_prime[r] < 1e-10:
#         d_star_prime[r] = 0

In [325]:
d_star_prime

array([  2.19503850e-02,   2.32286876e-05,   2.10492459e-04,
         6.45887990e-02,   5.88489462e-02,   5.53646056e-04,
         5.03637510e-05,   2.63766753e-04,   3.10523807e-01,
         2.72003631e-05,   6.67369644e-05,   7.77099465e-02,
         1.61202256e-05,   1.11554542e-04,   1.06545565e-02,
         2.43453364e-04,   8.59869665e-02,   8.14294996e-05,
         1.84730349e-02,   8.77350417e-02,   1.47655465e-02,
         1.48629430e-02,   2.01960128e-04,   2.26740894e-04,
         1.89494515e-04,   2.33031064e-04,   3.75070304e-05,
         4.64808432e-05,   1.40611974e-01,   9.07333757e-02])

In [326]:
d_star

array([  2.20992533e-02,   8.95300190e-57,   1.83126630e-56,
         6.48778373e-02,   5.96360778e-02,   5.01761181e-05,
         2.15554306e-06,   7.66050728e-06,   3.10623537e-01,
         1.26318788e-58,   3.95212974e-06,   7.75677290e-02,
         5.53945588e-58,   4.34877226e-58,   1.09411429e-02,
         4.71500474e-06,   8.60098913e-02,   1.28039625e-05,
         1.87151153e-02,   8.79759865e-02,   1.48729716e-02,
         1.49006407e-02,   3.16420618e-06,   8.12211778e-58,
         4.15077556e-57,   1.52046791e-06,   3.45824662e-06,
         5.91780319e-59,   1.40776460e-01,   9.09182184e-02])

In [327]:
d

array([[ 0.02211104],
       [ 0.        ],
       [ 0.        ],
       [ 0.06489338],
       [ 0.05962682],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.31062873],
       [ 0.        ],
       [ 0.        ],
       [ 0.07757486],
       [ 0.        ],
       [ 0.        ],
       [ 0.01097084],
       [ 0.        ],
       [ 0.08600981],
       [ 0.        ],
       [ 0.01872787],
       [ 0.08799176],
       [ 0.01486286],
       [ 0.0149007 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.14076784],
       [ 0.09093349]])

In [331]:
np.linalg.norm(P - np.dot(S, np.dot(np.asmatrix(d_star).T, c)))

1.6464414210627638

In [332]:
np.linalg.norm(P - np.dot(S, np.dot(np.asmatrix(d_star_prime).T, c)))

3.8449180522803235

In [330]:
np.asmatrix(d_star).T

matrix([[  2.20992533e-02],
        [  8.95300190e-57],
        [  1.83126630e-56],
        [  6.48778373e-02],
        [  5.96360778e-02],
        [  5.01761181e-05],
        [  2.15554306e-06],
        [  7.66050728e-06],
        [  3.10623537e-01],
        [  1.26318788e-58],
        [  3.95212974e-06],
        [  7.75677290e-02],
        [  5.53945588e-58],
        [  4.34877226e-58],
        [  1.09411429e-02],
        [  4.71500474e-06],
        [  8.60098913e-02],
        [  1.28039625e-05],
        [  1.87151153e-02],
        [  8.79759865e-02],
        [  1.48729716e-02],
        [  1.49006407e-02],
        [  3.16420618e-06],
        [  8.12211778e-58],
        [  4.15077556e-57],
        [  1.52046791e-06],
        [  3.45824662e-06],
        [  5.91780319e-59],
        [  1.40776460e-01],
        [  9.09182184e-02]])